In [722]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
import gc
import warnings
warnings.filterwarnings('ignore')
from impala.dbapi import connect
from impala.util import as_pandas
import pymysql
import ast
import requests
import random

In [723]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [757]:
class global_var:
    """ 全局参数配置 """
    # HIVE地址
    game_id = 1002113
    media_id = 45
    ad_account_id_group = np.array([6829, 6830])
    plan_num = 7

In [758]:
def get_game_id():
    conn = pymysql.connect(host='159.75.214.55', port=13306, user='tf',
                           passwd='tf!@#$')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1166 AND dev_game_id IS NOT NULL 
    '''
    cur.execute(sql)
    result_df = cur.fetchall()
    cur.close()
    conn.close()
    return result_df

In [759]:
# 获取近期优化计划的创意数据
def get_document():
    conn = pymysql.connect(host='159.75.214.55', port=13306, user='tf',
                           passwd='tf!@#$')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            pd.doc_content
        FROM
            db_ptom.ptom_plan_document pd 
        WHERE
            pd.game_ids IS NOT NULL 
            AND pd.game_ids != '' 
            AND FIND_IN_SET(1002113, pd.game_ids)
    '''
    cur.execute(sql)
    result = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    return result['doc_content'].values

In [727]:
# 获取score_image (分数大于550的image_id)
# def get_score_image():
#     conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
#                     password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
#     cursor = conn.cursor()
#     sql_engine = 'set hive.execution.engine=tez'
#     sql_queue = 'set tez.queue.name=offline'
#     sql = '''
#             SELECT
#             image_id,
#             label_ids 
#         FROM
#             dws.dws_image_score_d 
#         WHERE
#             dt = CURRENT_DATE 
#             AND media_id = 45 
#             AND image_create_role_roi >= 0.005 
#             AND ( image_create_role_retain_1d >= 0.01 OR image_create_role_retain_1d IS NULL ) 
#             AND score >= 500 
#         GROUP BY
#             image_id,
#             label_ids
#     '''
    
#     cursor.execute(sql_engine)
#     cursor.execute(sql_queue)
#     cursor.execute(sql)
#     result = as_pandas(cursor)
#     result['label_ids'] = result['label_ids'].astype(str)
#     result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
#     result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
# #     result = result[result['label_ids'].isin([27])]
# #     print(result)
#     # 关闭链接
#     cursor.close()
#     conn.close()
    
#     return result['image_id'].values

In [728]:
# def get_score_image_other():
#     conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
#                     password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
#     cursor = conn.cursor()
#     sql_engine = 'set hive.execution.engine=tez'
#     sql_queue = 'set tez.queue.name=offline'
#     sql = '''
#         SELECT
#             image_id,
#             label_ids 
#         FROM
#             dws.dws_image_score_d 
#         WHERE
#             dt = CURRENT_DATE 
#             AND media_id in (10,16,32) 
#             AND image_create_role_roi >= 0.005 
#             AND ( image_create_role_retain_1d >= 0.01 OR image_create_role_retain_1d IS NULL ) 
#             AND score >= 520 
#         GROUP BY
#             image_id,
#             label_ids
#     '''
#     cursor.execute(sql_engine)
#     cursor.execute(sql_queue)
#     cursor.execute(sql)
#     result = as_pandas(cursor)
#     result['label_ids'] = result['label_ids'].astype(str)
#     result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
#     result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
# #     result = result[result['label_ids'].isin([27])]
# #     print(result)
#     # 关闭链接
#     cursor.close()
#     conn.close()
    
#     return result['image_id'].values

In [783]:
# 获取近5天回款率高的素材
def get_good_image():
    conn = pymysql.connect(host='159.75.214.55', port=13306, user='tf',
                           passwd='tf!@#$')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            a.image_id AS 'image_id',
            a.image_name AS 'image_name',
            a.media_id,
            a.label_ids,
            ifnull( a.image_run_date_amount, 0 ) AS 'image_run_date_amount',
            ifnull( a.image_create_role_pay_num, 0 ) AS 'image_create_role_pay_num',
            ifnull( a.image_create_role_num, 0 ) AS 'image_create_role_num',
            ifnull( a.image_create_role_pay_sum, 0 ) AS 'image_create_role_pay_sum',
            ifnull( a.image_source_num, 0 ) AS 'image_source_num',
            (CASE WHEN ifnull( a.image_create_role_num, 0 )= 0 THEN
                    0 ELSE IFNULL( a.image_create_role_pay_num, 0 ) / ifnull( a.image_create_role_num, 0 ) 
                END ) AS 'image_create_role_pay_rate',
                (CASE WHEN ifnull( a.image_create_role_num, 0 )= 0 THEN
                        0 ELSE IFNULL( a.image_run_date_amount, 0 ) / ifnull( a.image_create_role_num, 0 ) 
                    END ) AS 'image_create_role_cost',
                    (CASE WHEN ifnull( a.image_create_role_pay_num, 0 )= 0 THEN
                            0 ELSE IFNULL( a.image_run_date_amount, 0 ) / ifnull( a.image_create_role_pay_num, 0 ) 
                        END ) AS 'image_create_role_pay_cost',
                        (CASE WHEN ifnull( a.image_run_date_amount, 0 )= 0 THEN
                                0 ELSE IFNULL( a.image_create_role_pay_sum, 0 ) / ifnull( a.image_run_date_amount, 0 ) 
                            END ) AS 'image_create_role_roi' 
                        FROM
                            (
                            SELECT
                                aa.image_id AS 'image_id',
                                aa.image_name AS 'image_name',
                                aa.media_id AS 'media_id',
                                aa.label_ids AS 'label_ids',
                                sum( aa.amount ) AS 'image_run_date_amount',
                                IFNULL( sum( aa.create_role_num ), 0 ) AS 'image_create_role_num',
                                IFNULL( sum( bb.pay_role_user_num ), 0 ) AS 'image_create_role_pay_num',
                                IFNULL( sum( bb.new_role_money ), 0 ) AS 'image_create_role_pay_sum',
                                IFNULL( sum( aa.image_source_num ), 0 ) AS 'image_source_num' 
                            FROM
                                (
                                SELECT
                                    a.game_id,
                                    a.channel_id,
                                    a.source_id,
                                    a.media_id,
                                    c.image_id,
                                    c.image_name,
                                    c.label_ids,
                                    IFNULL( sum( a.amount ), 0 ) AS amount,
                                    IFNULL( sum( create_role_num ), 0 ) AS create_role_num,
                                    count( DISTINCT b.plan_id ) AS 'image_source_num' 
                                FROM
                                    db_stdata.st_lauch_report a
                                    INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                    AND a.source_id = b.source_id 
                                    AND a.channel_id = b.chl_user_id
                                    LEFT JOIN db_data_ptom.ptom_image_info c ON b.image_id = c.image_id 
                                WHERE
                                    a.tdate_type = 'day' 
                                    AND a.tdate <= date( NOW()) AND a.tdate >= DATE_SUB( date( NOW()), INTERVAL 5 DAY ) 
                                    AND a.amount > 0 
                                    AND a.media_id IN ( 45 ) 
                                    AND b.image_id IS NOT NULL 
                                    AND b.image_id <> '' 
                                    AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1166 AND dev_game_id IS NOT NULL ) 
                                GROUP BY
                                    a.game_id,
                                    a.channel_id,
                                    a.source_id 
                                HAVING
                                    c.image_id IS NOT NULL 
                                    AND c.image_name <> '' 
                                ) aa
                                LEFT JOIN (
                                SELECT
                                    game_id,
                                    channel_id,
                                    source_id,
                                    IFNULL( sum( m.new_role_money ), 0 ) AS new_role_money,
                                    IFNULL( sum( m.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                FROM
                                    (
                                    SELECT
                                        a.game_id,
                                        a.channel_id,
                                        a.source_id,
                                        IFNULL( sum( a.new_role_money ), 0 ) AS new_role_money,
                                        IFNULL( sum( a.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                    FROM
                                        db_stdata.st_lauch_report a
                                        INNER JOIN db_data_ptom.ptom_plan pp ON ( a.game_id = pp.game_id AND a.channel_id = pp.chl_user_id AND a.source_id = pp.source_id ) 
                                    WHERE
                                        a.tdate = date(
                                        NOW()) 
                                        AND a.tdate_type = 'day' 
                                    GROUP BY
                                        a.game_id,
                                        a.channel_id,
                                        a.source_id 
                                    HAVING
                                        ( new_role_money > 0 OR pay_role_user_num > 0 ) UNION ALL
                                    SELECT
                                        c.game_id,
                                        c.channel_id,
                                        c.source_id,
                                        sum( c.create_role_money ) new_role_money,
                                        IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                    FROM
                                        db_stdata.st_game_days c 
                                    WHERE
                                        c.report_days = 5 
                                        AND c.tdate = date( NOW() - INTERVAL 24 HOUR ) 
                                        AND c.tdate_type = 'day' 
                                        AND c.query_type = 13 
                                    GROUP BY
                                        c.game_id,
                                        c.channel_id,
                                        c.source_id 
                                    HAVING
                                    ( new_role_money > 0 OR pay_role_user_num > 0 )) m 
                                GROUP BY
                                    game_id,
                                    channel_id,
                                    source_id 
                                ) bb ON aa.game_id = bb.game_id 
                                AND aa.channel_id = bb.channel_id 
                                AND aa.source_id = bb.source_id 
                            GROUP BY
                                aa.image_id,
                                aa.image_name,
                                aa.media_id 
                            HAVING
                            image_run_date_amount >= 100 
            ) a
            WHERE a.image_create_role_pay_sum >0 and 
                a.media_id = 45
                ORDER BY image_create_role_roi Desc
    '''
#     finalSql = sql.format(game_id)
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    result_df = result_df[['image_id','image_run_date_amount','image_create_role_pay_num','image_create_role_pay_sum',
                        'image_create_role_pay_cost','image_create_role_roi']]
    result_df = result_df[result_df['image_create_role_roi']>=0.05]
#     result_df['roi_rank'] = result_df['image_create_role_roi'].rank(ascending=False)
#     result_df = result_df[result_df['roi_rank'] <= np.ceil((result_df.shape[0] / 2))]
    return result_df['image_id'].values
#     return result_df

In [730]:
# AND image_name like "%刀%"
#                 AND create_time >='2022-08-17'

In [784]:
# 冷启动选素材
def get_image_cold():
    conn = pymysql.connect(host='159.75.214.55', port=13306, user='tf',
                           passwd='tf!@#$')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
            SELECT
            * 
            FROM
                db_data_ptom.ptom_image_info 
            WHERE
                NOT FIND_IN_SET( 45, exclude_media_ids ) 
                AND FIND_IN_SET(1002113,game_ids)
                AND image_style != 0
                '''
    cur.execute(sql)
    result = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    
    return result['image_id'].values

In [785]:
def filter_exclude_image(image_ids, media_id):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='159.75.214.55', port=13306, user='tf',
                           passwd='tf!@#$')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND NOT FIND_IN_SET({}, exclude_media_ids)
    '''
    result = pd.read_sql(sql.format(image_ids, media_id), conn)
    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [786]:
def filter_game_image(image_ids, game_id):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='159.75.214.55', port=13306, user='tf',
                           passwd='tf!@#$')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND FIND_IN_SET({}, game_ids)
    '''
    cur.execute(sql.format(image_ids, game_id))
    result = pd.read_sql(sql.format(image_ids, game_id), conn)

    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [787]:
def select_cold_image(game_id, media_id):
    """
    获取基础素材
    """
#     score_image_1 = get_score_image_other()
#     score_image_2 = get_score_image()
#     score_image_3 = get_old_image()
#     score_image = np.union1d(score_image_1,score_image_2)
#     score_image = filter_game_image(score_image, game_id)
#     if score_image.size == 0:
#         print('未匹配到素材，请检查素材评分')
#     if score_image.size <= 10:
#         score_image = np.union1d(score_image,np.random.choice(score_image_3,10)) 
    score_image = get_image_cold()
    score_image = filter_exclude_image(score_image, media_id)
#     score_image = np.setdiff1d(score_image,calculate_roi())
#     score_image = np.setdiff1d(score_image, get_lable_image())  # 过滤指定标签的素材
    print('匹配的素材ID', score_image)
    return score_image

In [788]:
def select_good_image(game_id, media_id):
    """
    获取基础素材
    """
#     score_image_1 = get_score_image_other()
#     score_image_2 = get_score_image()
#     score_image_3 = get_old_image()
#     score_image = np.union1d(score_image_1,score_image_2)
#     score_image = filter_game_image(score_image, game_id)
#     if score_image.size == 0:
#         print('未匹配到素材，请检查素材评分')
#     if score_image.size <= 10:
#         score_image = np.union1d(score_image,np.random.choice(score_image_3,10)) 
    score_image = get_good_image()
    if score_image.size != 0:
        score_image = filter_exclude_image(score_image, media_id)
#     score_image = np.setdiff1d(score_image,calculate_roi())
#     score_image = np.setdiff1d(score_image, get_lable_image())  # 过滤指定标签的素材
    print('匹配的素材ID', score_image)
    return score_image

In [765]:
# def select_image(game_id, media_id):
#     """
#     获取基础素材
#     """
# #     score_image_1 = get_score_image_other()
# #     score_image_2 = get_score_image()
# #     score_image_3 = get_old_image()
# #     score_image = np.union1d(score_image_1,score_image_2)
# #     score_image = filter_game_image(score_image, game_id)
# #     if score_image.size == 0:
# #         print('未匹配到素材，请检查素材评分')
# #     if score_image.size <= 10:
# #         score_image = np.union1d(score_image,np.random.choice(score_image_3,10)) 
    
    
#     score_image_1 = get_good_image()
#     score_image_1 = filter_exclude_image(score_image_1, media_id)
#     image_num = global_var.plan_num * global_var.ad_account_id_group.size / 2
#     if score_image_1.size <= image_num:
#         score_image_2 = get_image_cold()
#         score_image_2 = filter_exclude_image(score_image_2, media_id)
#         score_image_2 = np.setdiff1d(score_image_2,score_image_1)
#         score_image_2 = np.random.choice(score_image_2,int(image_num-score_image_1.size))
#         score_image = np.union1d(score_image_1,score_image_2)
#     else:
#         score_image = score_image_1

# #     score_image = np.setdiff1d(score_image,calculate_roi())
# #     score_image = np.setdiff1d(score_image, get_lable_image())  # 过滤指定标签的素材
#     print('匹配的素材ID', score_image)
#     return score_image

In [789]:
# 账号优先排斥素材
def get_priority_rejection():
    conn = pymysql.connect(host='159.75.214.55', port=13306, user='tf',
                           passwd='tf!@#$')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            aa.ad_account_id AS 'ad_account_id',
            aa.image_id AS 'image_id',
            aa.image_name AS 'image_name',
            aa.media_id AS 'media_id',
            aa.label_ids AS 'label_ids',
            sum( aa.amount ) AS 'image_run_date_amount',
            IFNULL( sum( aa.create_role_num ), 0 ) AS 'image_create_role_num',
            IFNULL( sum( bb.pay_role_user_num ), 0 ) AS 'image_create_role_pay_num',
            IFNULL( sum( bb.new_role_money ), 0 ) AS 'image_create_role_pay_sum',
            IFNULL( sum( aa.image_source_num ), 0 ) AS 'image_source_num' 
        FROM
            (
            SELECT
                a.game_id,
                a.channel_id,
                a.source_id,
                a.media_id,
                b.ad_account_id,
                c.image_id,
                c.image_name,
                c.label_ids,
                IFNULL( sum( a.amount ), 0 ) AS amount,
                IFNULL( sum( create_role_num ), 0 ) AS create_role_num,
                count( DISTINCT b.plan_id ) AS 'image_source_num' 
            FROM
                db_stdata.st_lauch_report a
                INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                AND a.source_id = b.source_id 
                AND a.channel_id = b.chl_user_id
                LEFT JOIN db_data_ptom.ptom_image_info c ON b.image_id = c.image_id 
            WHERE
                a.tdate_type = 'day' 
                AND a.tdate <= date( NOW()) AND a.tdate >= DATE_SUB( date( NOW()), INTERVAL 5 DAY ) 
                AND a.amount > 0 
                AND a.media_id IN ( 45 ) 
                AND b.image_id IS NOT NULL 
                AND b.image_id <> '' 
                AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
            GROUP BY
                a.game_id,
                a.channel_id,
                a.source_id 
            HAVING
                c.image_id IS NOT NULL 
                AND c.image_name <> '' 
            ) aa
            LEFT JOIN (
            SELECT
                game_id,
                channel_id,
                source_id,
                IFNULL( sum( m.new_role_money ), 0 ) AS new_role_money,
                IFNULL( sum( m.pay_role_user_num ), 0 ) AS pay_role_user_num 
            FROM
                (
                SELECT
                    a.game_id,
                    a.channel_id,
                    a.source_id,
                    IFNULL( sum( a.new_role_money ), 0 ) AS new_role_money,
                    IFNULL( sum( a.pay_role_user_num ), 0 ) AS pay_role_user_num 
                FROM
                    db_stdata.st_lauch_report a
                    INNER JOIN db_data_ptom.ptom_plan pp ON ( a.game_id = pp.game_id AND a.channel_id = pp.chl_user_id AND a.source_id = pp.source_id ) 
                WHERE
                    a.tdate = date(
                    NOW()) 
                    AND a.tdate_type = 'day' 
                GROUP BY
                    a.game_id,
                    a.channel_id,
                    a.source_id 
                HAVING
                    ( new_role_money > 0 OR pay_role_user_num > 0 ) UNION ALL
                SELECT
                    c.game_id,
                    c.channel_id,
                    c.source_id,
                    sum( c.create_role_money ) new_role_money,
                    IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                FROM
                    db_stdata.st_game_days c 
                WHERE
                    c.report_days = 5 
                    AND c.tdate = date( NOW() - INTERVAL 24 HOUR ) 
                    AND c.tdate_type = 'day' 
                    AND c.query_type = 13 
                GROUP BY
                    c.game_id,
                    c.channel_id,
                    c.source_id 
                HAVING
                ( new_role_money > 0 OR pay_role_user_num > 0 )) m 
            GROUP BY
                game_id,
                channel_id,
                source_id 
            ) bb ON aa.game_id = bb.game_id 
            AND aa.channel_id = bb.channel_id 
            AND aa.source_id = bb.source_id 
        GROUP BY
            aa.ad_account_id,
            aa.image_id,
            aa.image_name,
            aa.media_id 
        HAVING
            image_run_date_amount >= 100
    '''
#     finalSql = sql.format(game_id)
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    if result_df.shape[0] != 0:
        result_df['roi'] = result_df['image_create_role_pay_sum'] / result_df['image_run_date_amount']
        result_df['pay_cost'] = result_df.apply(lambda x: x.image_run_date_amount if x.image_create_role_pay_sum==0 else x.image_run_date_amount/x.image_create_role_pay_num, axis=1)
        # 优选条件
        df_1 = result_df[(result_df['pay_cost']<=2000)&(result_df['roi']>=0.1)&(result_df['image_create_role_pay_sum']>=100)&(result_df['image_source_num']<=10)][['ad_account_id','image_id']]
        # 排斥条件
        df_2 = result_df[(result_df['image_run_date_amount']>=2500)&(result_df['image_create_role_pay_sum']==0)][['ad_account_id','image_id']] 
        return df_1, df_2
    else:
        print('数据为空')
    

In [790]:
def get_ad_create(plan_result):
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.iloc[i].to_json()))
#     open_api_url_prefix = "https://ptom-pre.caohua.com/"
    open_api_url_prefix = "https://biz-ptom.caohua.com/"
    uri = "model/generationPlanBatchTask"
    url = open_api_url_prefix + uri
    params = {
        "secretkey": "abc2018!@**@888",
        "mediaId": 45
    }
    rsp = requests.post(url, json=ad_info, params=params)
    rsp_data = rsp.json()
    print('结束....')
    return rsp_data

In [791]:
image = select_cold_image(global_var.game_id, global_var.media_id)

匹配的素材ID [   46    49    51    53    54    55    62    63    71    72    73    74
    90    94    95    97    98   102   103   104   105   106   107   108
   109   111   112   113   115   116   118   120   121   122   123   135
   136 48343 48344 48345 48346 48347 48348 48349 48350 48351 48352]


In [792]:
select_good_image(global_var.game_id, global_var.media_id)

匹配的素材ID [62]


array([62], dtype=int64)

In [793]:
try:
    df_1, df_2 = get_priority_rejection()
except:
    df_1 = pd.DataFrame(columns=['ad_account_id','image_id'])
    df_2 = pd.DataFrame(columns=['ad_account_id','image_id'])

数据为空


In [794]:
df_1

,ad_account_id,image_id


In [795]:
df_2

,ad_account_id,image_id


In [796]:
# 每个账号分配素材并建计划
plan_result = pd.DataFrame()
image_3 = select_good_image(global_var.game_id, global_var.media_id)
for ad_account_id in global_var.ad_account_id_group:
    # 该账号素材 image_1(账号优先素材)，image_2(冷启动素材)，image_3(全局优先素材)
    image_1 = df_1[df_1['ad_account_id']==ad_account_id]['image_id'].values  
    image_3 = np.setdiff1d(image_3, image_1)
    image_3 = np.setdiff1d(image_3, df_2[df_2['ad_account_id']==ad_account_id]['image_id'].values)
    image_2 = np.setdiff1d(image, df_2[df_2['ad_account_id']==ad_account_id]['image_id'].values)
    image_2 = np.setdiff1d(image_2, image_1)
    image_2 = np.setdiff1d(image_2, image_3)
    if image_1.size>=global_var.plan_num:
        image_group = np.random.choice(image_1, size=global_var.plan_num, replace=False)
    elif (image_1.size<global_var.plan_num) & (image_3.size >=(global_var.plan_num - image_1.size)//2):
        image_group =np.random.choice(image_3, size=(global_var.plan_num - image_1.size)//2, replace=False)
        image_group = np.union1d(image_group,image_1)
        if image_2.size >= global_var.plan_num-image_group.size:
            image_group =  np.union1d(image_group,np.random.choice(image_2, size=(global_var.plan_num-image_group.size), replace=False))
        else:
            image_group =  np.union1d(image_group,image_2)
    else:
        image_group =np.union1d(image_3,image_1)
        if image_2.size >= global_var.plan_num-image_group.size:
            image_group =  np.union1d(image_group,np.random.choice(image_2, size=(global_var.plan_num-image_group.size), replace=False))
        else:
            image_group =  np.union1d(image_group,image_2)
    
    for image_id in image_group:
        temp = pd.DataFrame({'ad_account_id': [ad_account_id], 'image_id': [image_id]})
        plan_result = plan_result.append(temp)
plan_result = plan_result.reset_index(drop=True)

匹配的素材ID [62]


In [744]:
# 每个账号分配素材并建计划
# plan_result = pd.DataFrame()
# for ad_account_id in global_var.ad_account_id_group:
#     # 该账号素材 image_1(账号优先素材)，image_2(冷启动素材)，image_3(全局优先素材)
#     image_1 = df_1[df_1['ad_account_id']==ad_account_id]['image_id'].values
    
#     image_2 = np.setdiff1d(image, df_2[df_2['ad_account_id']==ad_account_id]['image_id'].values)
#     image_2 = np.setdiff1d(image_2, image_1)
#     if image_1.size>=global_var.plan_num:
#         image_group = np.random.choice(image_1, size=global_var.plan_num, replace=False)
#     else:
#         if image_2.size >= global_var.plan_num-image_1.size:
#             image_group =  np.union1d(image_1,np.random.choice(image_2, size=(global_var.plan_num-image_1.size), replace=False))
#         else:
#             image_group =  np.union1d(image_1,image_2)
#     for image_id in image_group:
#         temp = pd.DataFrame({'ad_account_id': [ad_account_id], 'image_id': [image_id]})
#         plan_result = plan_result.append(temp)
# plan_result = plan_result.reset_index(drop=True)

In [797]:
plan_result

,ad_account_id,image_id
0,6829,49
1,6829,62
2,6829,104
3,6829,108
4,6829,109
5,6829,118
6,6829,48344
7,6830,62
8,6830,103
9,6830,104


In [798]:
# 基础固定参数
plan_result['producttypes'] = plan_result.apply(lambda x:[], axis=1)
plan_result['ftypes'] = plan_result.apply(lambda x:[1, 2], axis=1)   # 版位
plan_result['game_id'] = global_var.game_id
plan_result['pause'] = plan_result.apply(lambda x:False, axis=1)     # 是否暂停 
plan_result['deepTransTypes'] = plan_result.apply(lambda x: [26],axis=1)   # 转化参数
# plan_result['plan_auto_task_id'] = "12641,12639,12640"    # TODO
plan_result['plan_auto_task_id'] = "1,2"
plan_result['op_id'] = 13268
plan_result['subject'] = 3   # 跑安卓
plan_result['flag'] = 'BD'
plan_result['game_name'] = '热血'

In [799]:
df_kw = pd.read_excel('../百度热血意图词.xls')
# df_kw = df_kw.dropna(axis=0,subset=['SLG意图词'])

In [800]:
# 定向 
plan_result['customAge'] = np.nan
df_kw = pd.read_excel('../百度热血意图词.xls')
words_1 = ",".join(np.union1d(list(map(str,df_kw.dropna(axis=0,subset=['A'])['A'].values)),list(map(str,np.random.choice(df_kw.dropna(axis=0,subset=['意图词'])['意图词'].values, 1600, replace=False)))))
words_2 = ",".join(np.union1d(list(map(str,df_kw.dropna(axis=0,subset=['B'])['B'].values)),list(map(str,np.random.choice(df_kw.dropna(axis=0,subset=['意图词'])['意图词'].values, 1600, replace=False)))))
words_3 = ",".join(np.union1d(list(map(str,df_kw.dropna(axis=0,subset=['C'])['C'].values)),list(map(str,np.random.choice(df_kw.dropna(axis=0,subset=['意图词'])['意图词'].values, 1600, replace=False)))))
words_4 = ",".join(np.union1d(list(map(str,df_kw.dropna(axis=0,subset=['D'])['D'].values)),list(map(str,np.random.choice(df_kw.dropna(axis=0,subset=['意图词'])['意图词'].values, 1600, replace=False)))))
words_5 = ",".join(map(str,np.random.choice(df_kw.dropna(axis=0,subset=['意图词'])['意图词'].values, 1900, replace=False)))
plan_result['keywords'] = plan_result.apply(lambda x:np.random.choice([words_1,words_2,words_3,words_4,words_5],p=[0.2,0.2,0.2,0.2,0.2]), axis=1)
plan_result['device'] = 2   # 安卓
plan_result['age'] = plan_result.apply(lambda x:np.random.choice(["2,3,4,5"]), axis=1)
plan_result['sex'] = plan_result.apply(lambda x:np.random.choice([2,0],p=[0.95,0.05]), axis=1)    # 男性为主
plan_result['autoExpansion'] = 0
plan_result['education'] = 0
plan_result['keywordsExtend'] = 1
plan_result['newInterests'] = 0
plan_result['iosVersion'] = 0
plan_result['androidVersion'] = plan_result.apply(lambda x:np.random.choice(["0","8","9","10"],p=[0.2,0.2,0.3,0.3]), axis=1)           # 安卓版本
plan_result['excludeTrans'] = plan_result.apply(lambda x:np.random.choice(["0","1","4"],p=[0.2,0.2,0.6]), axis=1)              # 排除人群
plan_result['excludeCrowd'] = plan_result.apply(lambda x: "126971537", axis=1)              # 排除人群包
plan_result['region'] = "2000,3000,4000,5000,8000,9000,10000,11000,12000,13000,14000,15000,16000,17000,18000,19000,20000,21000,22000,23000,24000,25000,26000,27000,28000,29000,30000,31000,32000,33000"
plan_result['androidBrands'] = plan_result.apply(lambda x:np.random.choice(["0","3","1,2,3","1,2,3,4,6,8"],p=[0.2,0.4,0.3,0.1]), axis=1)
plan_result['mobilePhonePrice'] =  plan_result.apply(lambda x:np.random.choice(["0","2,3,4,5","3,4,5","4,5"],p=[0.2,0.3,0.3,0.2]), axis=1)

In [801]:
# 出价方式
plan_result['transFrom'] = 1
plan_result['payMode'] = 1
plan_result['ocpcLevel'] = 2
plan_result['isSkipStageOne'] = True
    
plan_result['optimizeDeepTrans'] = plan_result.apply(lambda x:np.random.choice([False,True],p=[0.9,0.1]), axis=1)
# plan_result['optimizeDeepTrans'] = plan_result['optimizeDeepTrans'].apply(lambda x: False)
plan_result['transType'] = plan_result['optimizeDeepTrans'].apply(lambda x:26 if x==False else 4)
plan_result['ocpcBid'] = plan_result['optimizeDeepTrans'].apply(lambda x:random.randint(1800.00, 1900.00) if x==False else random.randint(150.00, 160.00))
plan_result['deepOcpcBid'] = plan_result['optimizeDeepTrans'].apply(lambda x:random.randint(1500.00, 1600.00) if x==True else np.nan)
plan_result['useRoi'] = plan_result['optimizeDeepTrans'].apply(lambda x:True if x==False else np.nan)
plan_result['roiRatio'] = plan_result['optimizeDeepTrans'].apply(lambda x:0.1 if x==False else np.nan)
plan_result['deepTransType'] = plan_result['optimizeDeepTrans'].apply(lambda x:np.nan if x==False else 26)

In [802]:
ad_info = []
for i in range(plan_result.shape[0]):
    ad_info.append(json.loads(plan_result.loc[i, ['autoExpansion','education', 'customAge', 'keywords',
       'keywordsExtend', 'newInterests', 'sex', 'iosVersion', 'androidVersion',
       'excludeTrans', 'region', 'device', 'age','androidBrands','mobilePhonePrice']].to_json()))
plan_result['audience'] = ad_info
plan_result.drop(['autoExpansion','education', 'customAge', 'keywords',
       'keywordsExtend', 'newInterests', 'sex', 'iosVersion', 'androidVersion',
       'excludeTrans', 'region', 'device', 'age','androidBrands','mobilePhonePrice'], axis=1, inplace=True)
ad_info = []
for i in range(plan_result.shape[0]):
    ad_info.append(json.loads(plan_result.loc[i, [ 'transType', 'ocpcLevel',
       'isSkipStageOne', 'payMode', 'optimizeDeepTrans', 'transFrom',
       'ocpcBid', 'deepOcpcBid', 'deepTransType', 'roiRatio', 'useRoi']].to_json()))
plan_result['ocpc'] = ad_info
plan_result.drop([ 'transType', 'ocpcLevel',
       'isSkipStageOne', 'payMode', 'optimizeDeepTrans', 'transFrom',
       'ocpcBid', 'deepOcpcBid', 'deepTransType', 'roiRatio', 'useRoi'], axis=1, inplace=True)
plan_result['schedule'] = plan_result['ocpc'].apply(lambda x: [   {
                                                                        "weekDay": 1,
                                                                        "startHour": 0,
                                                                        "endHour": 9
                                                                    },
                                                                    {
                                                                        "weekDay": 2,
                                                                        "startHour": 0,
                                                                        "endHour": 9
                                                                    },
                                                                    {
                                                                        "weekDay": 3,
                                                                        "startHour": 0,
                                                                        "endHour": 9
                                                                    },
                                                                    {
                                                                        "weekDay": 4,
                                                                        "startHour": 0,
                                                                        "endHour": 9
                                                                    },
                                                                    {
                                                                        "weekDay": 5,
                                                                        "startHour": 0,
                                                                        "endHour": 9
                                                                    },
                                                                    {
                                                                        "weekDay": 6,
                                                                        "startHour": 0,
                                                                        "endHour": 9
                                                                    },
                                                                    {
                                                                        "weekDay": 7,
                                                                        "startHour": 0,
                                                                        "endHour": 9
                                                                    }
                                                                ])

In [803]:
subtitle_array = np.array(['全新竖版热血武侠RPG','我的大刀四十米，你的呢','30级转职,50级血脉觉醒','开局十连抽,装备全靠爆','经典热血武侠,兄弟一起玩','10倍爆率，礼包码dalao666','十倍爆率刀刀暴击装备满地','影剑江湖侠义不灭热血不改','侠客闯江湖，战力不封顶'])

In [804]:
# plan_result['bgtctltype'] = 2  # 加速投放
# 创意
document = get_document()
plan_result['materialstyle'] = 113
plan_result['brand'] = plan_result.apply(lambda x: '热血群侠传2' ,axis=1)
#     plan_result['subtitle'] = '下载领取中国区专属福利！'
plan_result['subtitle'] = plan_result.apply(lambda x: np.random.choice(subtitle_array,1)[0], axis=1)
plan_result['userPortrait'] = "https://feed-image.baidu.com/0/pic/-455349395_969751491_-485081684.jpg"
plan_result['titles'] = plan_result.apply(lambda x: np.random.choice(get_document(),1), axis=1)
plan_result['image_ids'] = plan_result['image_id'].apply(lambda x: [x])
plan_result.drop('image_id', axis=1, inplace=True)
plan_result.to_csv('./plan_result.csv', index=0)  # 保存创建日志

In [806]:
print('计划数量%d' % plan_result.shape[0])
rsp_data = get_ad_create(plan_result)
print(rsp_data)

计划数量14
结束....
{'code': 200, 'data': True, 'msg': ''}


In [805]:
plan_result['image_ids']

0        [49]
1        [62]
2       [104]
3       [108]
4       [109]
5       [118]
6     [48344]
7        [62]
8       [103]
9       [104]
10      [108]
11      [135]
12    [48345]
13    [48352]
Name: image_ids, dtype: object